## Bioaerosol transmission
We consider 2 scenarios: wearing a conventional shield ('normal'), and wearing a non-conventional shield that has both bottom & side surfaces added ('novel'). The breathing rate thru the nostrils of this person is set to a sinusoidal function, while the airflow generated by speech is set as a semi-random flow rate (see our OpenFOAM case's boundary conditions for more details).

### Inhaled bioaerosols trajectories
We now visualise only the bioaerosols that are inhaled by the shield-wearing person. The bioaerosol's trajectories between their initial exhalation from the mouth to the inhalation thru the nostrils are visualised using linear lines that connect bioaerosols' locations (snapshot at particular times) generated by OpenFOAM. Note, only a random subset of total bioaerosols inhaled are visualised.

In [16]:
# Extract bioaerosol data from computation #1

import pyvista
import math

# Read geometry
fileName = "./case1.5Normal/case.foam"
readerMesh = pyvista.OpenFOAMReader(fileName)
mesh = readerMesh.read()
meshBoundaries = mesh["boundary"]
meshPerson = meshBoundaries['person']

# ID's of all particles that are inhaled
writeInterval = 1
times = range(writeInterval, 91, writeInterval)
IDs = []
for time in times:
    if (time % 2) == 0:
        time = math.trunc(time / 2)
    else:
        time = time / 2
    path = "./case1.5Normal/postProcessing/lagrangian/cloud/inhaledNose/"
    #timeStr = str(time)
    timeStr = f"{time}"
    fileParticlesNostrils = path + timeStr + "/nostrils.post"
    with open(fileParticlesNostrils) as f:
        line = f.readline()
        while line:
            try:
                line = f.readline()
                lineSplit = line.split()
                ID = lineSplit[14]
                IDs.append(ID)
            except IndexError:
                f.close()
                break

# ID's of a subset of all inhaled particles
import random
IDsSubset = []
l = int(len(IDs) / 50)
for i in range(0, l):
    randInt = random.randrange(0, len(IDs))
    IDsSubset.append(int(IDs[randInt]))

# Thanks Cory: https://public.kitware.com/pipermail/vtkusers/2017-March/098260.html
import vtk
import numpy as np
reader = vtk.vtkPolyDataReader()

# get positions of all particles at every time-step
timesVtk = range(2500, 91500, 1000)
particlesPosSubsetRaw = {}
for timeStepVtk in timesVtk:
    pathVtk = "./case1.5Normal/VTK/lagrangian/cloud/cloud_" + f"{timeStepVtk}" + ".vtk"
    reader.SetFileName(pathVtk)
    reader.Update()
    
    # Store data in numpy array: https://www.kitware.com/improved-vtk-numpy-integration-part-2/
    from vtk.numpy_interface import dataset_adapter as dsa
    timeData = dsa.WrapDataObject(reader.GetOutput())
    
    # find index of subset particle's ID
    timeParticlesPos = timeData.GetPoints()
    #mydata.PointData.keys()
    IDsAll =  timeData.PointData["origId"] 
    indexes = []
    for IDsSubset_i in IDsSubset:
        try:
            index = np.where(IDsAll == IDsSubset_i)[0][0]
        except IndexError:
            continue
        indexes.append(index)

    particlesPosSubsetTime_i = {}
    for index in indexes:
        
        particlesPosSubsetTime_i[f"{(int(IDsAll[index]))}"] = timeParticlesPos[index]
        realTime = (timeStepVtk - 500) * 0.0005
        particlesPosSubsetRaw[f"{(realTime)}"] = particlesPosSubsetTime_i

# restructure into single particle ID's over time
particlesPosSubset = {}
for IDsSubset_i in IDsSubset:
    dictionary = {}
    for time in times:
        time = time / 2
        try:
            dictionary[f"{(time)}"] = particlesPosSubsetRaw[f"{(time)}"][f"{(IDsSubset_i)}"]
        except KeyError:
            continue
    particlesPosSubset[f"{(IDsSubset_i)}"] = dictionary

In [21]:
# Extract bioaerosol data from computation #2

# Read geometry
fileName2 = "./case1.5Enclosed/case.foam"
readerMeshEnc = pyvista.OpenFOAMReader(fileName2)
meshEnc = readerMeshEnc.read()
meshBoundariesEnc = meshEnc["boundary"]
meshPersonEnc = meshBoundariesEnc['person']

# ID's of all particles that are inhaled
IDsEnc = []
for time in times:
    if (time % 2) == 0:
        time = math.trunc(time / 2)
    else:
        time = time / 2
    timeStr = f"{time}"
    path2 = "./case1.5Enclosed/postProcessing/lagrangian/cloud/inhaledNose/"
    fileParticlesNostrils2 = path2 + timeStr + "/nostrils.post"
    with open(fileParticlesNostrils2) as f2:
        lineEnc = f2.readline()
        while lineEnc:
            try:
                lineEnc = f2.readline()
                lineSplitEnc = lineEnc.split()
                IDsEnc.append(lineSplitEnc[14])
            except IndexError:
                break

# ID's of a subset of all inhaled particles
IDsSubsetEnc = []
lEnc = int(len(IDsEnc) / 50)
for i in range(0, lEnc):
    randInt = random.randrange(0, len(IDsEnc))
    IDsSubsetEnc.append(int(IDsEnc[randInt]))

# get positions of all particles at every time-step
particlesPosSubsetRawEnc = {}
for timeStepVtk in timesVtk:
    pathVtk = "./case1.5Enclosed/VTK/lagrangian/cloud/cloud_" + f"{timeStepVtk}" + ".vtk"
    reader.SetFileName(pathVtk)
    reader.Update()
    
    # Store data in numpy array: https://www.kitware.com/improved-vtk-numpy-integration-part-2/
    from vtk.numpy_interface import dataset_adapter as dsa
    timeData = dsa.WrapDataObject(reader.GetOutput())
    
    # find index of subset particle's ID
    timeParticlesPos = timeData.GetPoints()
    #mydata.PointData.keys()
    IDsAll =  timeData.PointData["origId"] 
    indexes = []
    for IDsSubset_i in IDsSubsetEnc:
        try:
            index = np.where(IDsAll == IDsSubset_i)[0][0]
        except IndexError:
            continue
        indexes.append(index)

    particlesPosSubsetTime_i = {}
    for index in indexes:
        
        particlesPosSubsetTime_i[f"{(int(IDsAll[index]))}"] = timeParticlesPos[index]
        realTime = (timeStepVtk - 500) * 0.0005
        particlesPosSubsetRawEnc[f"{(realTime)}"] = particlesPosSubsetTime_i

# restructure into single particle ID's over time
particlesPosSubsetEnc = {}
for IDsSubset_i in IDsSubsetEnc:
    dictionary = {}
    for time in times:
        time = time / 2
        try:
            dictionary[f"{(time)}"] = particlesPosSubsetRawEnc[f"{(time)}"][f"{(IDsSubset_i)}"]
        except KeyError:
            continue
    particlesPosSubsetEnc[f"{(IDsSubset_i)}"] = dictionary

2025-05-02 01:35:04.164 (1206.126s) [    7F9531347740]  vtkOpenFOAMReader.cxx:9270  WARN| vtkOpenFOAMReaderPrivate (0x55bdecc01fa0): boundaryField nonConformalCyclic_on_iFaceDownCube not found in object U at time = 0
2025-05-02 01:35:04.231 (1206.193s) [    7F9531347740]  vtkOpenFOAMReader.cxx:9270  WARN| vtkOpenFOAMReaderPrivate (0x55bdecc01fa0): boundaryField nonConformalCyclic_on_iFaceDownCube not found in object k at time = 0
2025-05-02 01:35:04.296 (1206.258s) [    7F9531347740]  vtkOpenFOAMReader.cxx:9270  WARN| vtkOpenFOAMReaderPrivate (0x55bdecc01fa0): boundaryField nonConformalCyclic_on_iFaceDownCube not found in object nut at time = 0
2025-05-02 01:35:04.359 (1206.322s) [    7F9531347740]  vtkOpenFOAMReader.cxx:9270  WARN| vtkOpenFOAMReaderPrivate (0x55bdecc01fa0): boundaryField nonConformalCyclic_on_iFaceDownCube not found in object p at time = 0


In [25]:
# Pre-process bioaerosols' data then visualise

import pyvista, numpy as np
plotter = pyvista.Plotter(shape=(2, 1))

arrayParticlePos = np.array([[0.0,0.0,0.0]])
for ID in IDsSubset:
    arrayParticlePositionTimes = np.array([[0,0,0]])
    timesSubset = particlesPosSubset[f"{(ID)}"].keys()
    for time in timesSubset:
        position = particlesPosSubset[f"{(ID)}"][time]
        arrayPosition = np.array([[position[0], position[1], position[2]]])
        arrayParticlePositionTimes = np.append(arrayParticlePositionTimes, arrayPosition, axis=0)
    arrayParticlePositionTimes = np.delete(arrayParticlePositionTimes, 0, 0)
    
    points = arrayParticlePositionTimes
    plotter.subplot(0, 0)
    plotter.add_lines(points, color='red', width=1, connected=True)
plotter.add_mesh(meshPerson, color='pink', line_width=3)

arrayParticlePos = np.array([[0.0,0.0,0.0]])
for IDEnc in IDsSubsetEnc:
    arrayParticlePositionTimes = np.array([[0,0,0]])
    timesSubset = particlesPosSubsetEnc[f"{(IDEnc)}"].keys()
    for time in timesSubset:
        position = particlesPosSubsetEnc[f"{(IDEnc)}"][time]
        arrayPosition = np.array([[position[0], position[1], position[2]]])
        arrayParticlePositionTimes = np.append(arrayParticlePositionTimes, arrayPosition, axis=0)
    arrayParticlePositionTimes = np.delete(arrayParticlePositionTimes, 0, 0)
    
    points = arrayParticlePositionTimes
    plotter.subplot(1, 0)
    plotter.add_lines(points, color='red', width=1, connected=True)
plotter.add_mesh(meshPersonEnc, color='pink', line_width=3)

plotter.link_views()
plotter.show()

Widget(value='<iframe src="/proxy/34803/index.html?ui=P_0x7f94e4793150_1&reconnect=auto" class="pyvista" style…

For the normal shield, the concentration of bioaerosols appears greater towards the centre of the shield (in a horizontal plane) resulting in less inhaled at larger distances away (nearer the shield's sides in other words), with almost all bioaerosols inhaled passing the shield's bottom edge within half the width. For the ‘novel’ shield, significantly less bioaerosols are inhaled as compared to when wearing a normal shield (more details, perhaps?).

### Discussion
We content that infection of COVID-type virus is very likely when a person (who is not wearing a face mask or shield) is speaking to another person (that is wearing face shield) during a short conversation. Our computational predicted time for this occuring is much shorter than the 5-to-30 minutes estimated elsewhere [https://www.nature.com/articles/s41598-023-47829-8]. We also contend that a face shield modified to be almost fully enclosed will reduce this transmission risk of ~ 50%. Moreover, we have high confidence this computationally-derrived research finding is confirmation that the design of face shields can be improved significanly.

Our non-vaccine approach of disease intervention is not novel, other groups are looking at potential mitigation using material approaches [https://pmc.ncbi.nlm.nih.gov/articles/PMC10190935/]. Our fluid-dynamic approach may complement theirs to form a more integrated defense mechanism. Further, our fluid-dynamic approach may be more effective than vaccines against viruses that have demonstrated the ability to evade vaccine-triggered responses, such as COVID-Omicron [https://pmc.ncbi.nlm.nih.gov/articles/PMC9047469/].